In [1]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import datasets
import  os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
# x: [60k, 28, 28]
# y: [60k]

(x, y),_ = datasets.mnist.load_data()

In [3]:
# 把 x的数值范围从 [0 ~ 255] 改为 [0 ~ 1]
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32)

In [4]:
print(x.shape, y.shape, x.dtype, y.dtype)

(60000, 28, 28) (60000,) <dtype: 'float32'> <dtype: 'int32'>


In [5]:
#  数据范围， 看 x y 的最小值和最大值
print(tf.reduce_min(x), tf.reduce_max(x))

tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [6]:
print(tf.reduce_min(y), tf.reduce_max(y))

tf.Tensor(0, shape=(), dtype=int32) tf.Tensor(9, shape=(), dtype=int32)


In [7]:
# 每次取128 张照片
train_db = tf.data.Dataset.from_tensor_slices((x, y)).batch(128)
train_iter = iter(train_db)
sample = next(train_iter)
print('batch:', sample[0].shape, sample[1].shape )

batch: (128, 28, 28) (128,)


In [12]:
# [b, 784] => [b, 256] => [b, 128] => [b, 10]
# [dim_in, dim_out], [dim_out]
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

In [13]:
lr = 1e-3

In [15]:
# h1 = x @ W1 + b1

for epoch in range(10): # iterate db for 10
    for step, (x, y) in enumerate(train_db): # for every batch
        # x: [128, 28, 28]
        # y: [128]

        # x: [b, 28, 28] => [b, 28* 28]
        x = tf.reshape(x, [-1, 28* 28])

        with tf.GradientTape() as tape: # tf.Variable

            # h1 = x @ w1 + b1
            # [b, 784] @ [784, 256] + [256] => [b, 256] + [256]
            h1 = x@w1 + tf.broadcast_to(b1, [x.shape[0], 256])
            h1 = tf.nn.relu(h1)
            # [b, 256] => [b, 128]
            h2 = h1@w2 + b2
            h2 = tf.nn.relu(h2)
            # [b, 128] => [b, 10]
            out = h2@w3 + b3

            # compute loss
            # out: [b, 10]
            # y: [b]
            y_onehot = tf.one_hot(y, depth=10)

            # mse = mean(sum((y-out)^2))
            # [b, 10]
            loss = tf.square(y_onehot - out)
            # mean: scalar
            loss = tf.reduce_mean(loss)

        # compute gradients
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])

        # print(grads)

        # w1 = w1 - lr * w1_grad
        w1.assign_sub(lr * grads[0])
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])

        if step % 100 == 0:
            print(epoch, step, 'loss', float(loss))


0 0 loss 0.15757174789905548
0 100 loss 0.15880440175533295
0 200 loss 0.16019687056541443
0 300 loss 0.14574292302131653
0 400 loss 0.14883914589881897
1 0 loss 0.13481266796588898
1 100 loss 0.1387593150138855
1 200 loss 0.13869956135749817
1 300 loss 0.12776817381381989
1 400 loss 0.1326252669095993
2 0 loss 0.11999980360269547
2 100 loss 0.12534889578819275
2 200 loss 0.12428710609674454
2 300 loss 0.1154339462518692
2 400 loss 0.1214115172624588
3 0 loss 0.10942494869232178
3 100 loss 0.11567869037389755
3 200 loss 0.11386444419622421
3 300 loss 0.10653138160705566
3 400 loss 0.11323828995227814
4 0 loss 0.10151724517345428
4 100 loss 0.10835667699575424
4 200 loss 0.10601433366537094
4 300 loss 0.09979565441608429
4 400 loss 0.10690031200647354
5 0 loss 0.09528177976608276
5 100 loss 0.10252662003040314
5 200 loss 0.09984661638736725
5 300 loss 0.09449604898691177
5 400 loss 0.10174831002950668
6 0 loss 0.09021513909101486
6 100 loss 0.0977613776922226
6 200 loss 0.09487023204565